In [2]:
import math
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Input Test A and B sensitivities and specificities and disease prevalance
## input prevalance as a comma separated list of possibles to be evaluated

suggestions: layton = 0.052, ut % positive tests = 0.046, salt lake high = .399

In [3]:
#characteristics of the tests
sensA = .63
specA = .99
sensB = .841
specB = .923

#disease prevalance in the population
### Layton is at .0005%, positive tests in UT = 4.5%, salt lake high = .415%
# for example prev = (.050,.010,.1)
prev_list = (.00052,.00415,.045)

pop=350

### for reference, here are the sensitivies and specificities used earlier
#pcr option 1
sen_pcr1 = .63
sen_pcr2 = .73
spec_pcr = .99
#serology
sen_ser = .841
spec_ser = .923

# Ignore this code

In [7]:
#Calculations and Output generation, saves as .csv
Test_characteristics = pd.DataFrame(columns=('Prevalance','Test Name','sensitivity','specificitiy','negPredVal'))
for prev in prev_list:
    
    POsens=sensA+sensB -(sensA*sensB)
    POspec = specA*specB
    POnpv = (POspec*(1-prev))/((1-POsens)*prev+POspec*(1-prev))

    PAsens = sensA*sensB
    PAspec = specA+specB-(specA*specB)
    PAnpv = (PAspec*(1-prev))/((1-PAsens)*prev+PAspec*(1-prev))

    SOsens = sensA+(1-sensA)*sensB
    SOspec = specA*specB
    SOnpv = (SOspec*(1-prev))/((1-SOsens)*prev+SOspec*(1-prev))

    SAsens = sensA*sensB
    SAspec = specA+(1-specA)*specB
    SAnpv = (SAspec*(1-prev))/((1-SAsens)*prev+SAspec*(1-prev))

    npvA=(specA*(1-prev))/((1-sensA)*prev+specA*(1-prev))
    npvB=(specB*(1-prev))/((1-sensB)*prev+specB*(1-prev))
    #build lists for dataframe
    prevalance = [prev]*6
    sensitivities = [sensA,sensB,POsens,PAsens,SOsens,SAsens]
    specificities = [specA,specB,POspec,PAspec,SOspec,SAspec]
    negPredVals = [npvA,npvB,POnpv,PAnpv,SOnpv,SAnpv]
    names = ['Test A','Test B','Parallel OR','Parallel AND','Series OR','Series AND']
    data_tuples = list(zip(prevalance,names,sensitivities,specificities,negPredVals))
    Test_Comparison = pd.DataFrame(data_tuples, columns=('Prevalance','Test Name','sensitivity','specificitiy','negPredVal'))
    Test_characteristics = Test_characteristics.append(Test_Comparison)
Test_characteristics.to_csv('Test_Char')

## Combining Tests
### Parallel - done at same time, interp together
#### OR test
##### yields a positive diagnosis if either test is positive and a negative diagnosis if both tests are negative

Sens = SEa + SEb - (SEa × SEb)

Spec = SPa × SPb

#### AND test
##### yields a positive diagnosis only if both tests are positive and a negative diagnosis if either test is negative


Sens = SEa × SEb

Spec = SPa + SPb - (SPa × SPb)

### Series - done one at a time, 1st determines if 2nd done
#### OR test
##### if the first test is positive, the diagnosis is positive; otherwise, the second test is performed. If the second test is positive after a negative first test, then the diagnosis also is positive; otherwise, the diagnosis is negative


Sens = SEa + (1 - SEa) × SEb

Spec = SPa × SPb

#### AND test
##### if the first test is positive, the second test is performed. If the second test is positive, the diagnosis is positive; otherwise, the diagnosis is negative

Sens = SEa × SEb

Spec = SPa + (1 - SPa) × SPb

## Negative Predictive Value
#### What is the likelihood that this patient does not have the disease when the test result is negative?
The probability that the disease is absent given the pretest probability (prevalance) and negative test result

NPV = (Spec × (1-prev)) / (Spec × (1-prev) + (1-Sens) × prev




Clinical Evaluation of Diagnostic Tests
Susan Weinstein, Nancy A. Obuchowski and Michael L. Lieber
American Journal of Roentgenology. 2005;184: 14-19. 10.2214/ajr.184.1.01840014

Read More: https://www.ajronline.org/doi/abs/10.2214/ajr.184.1.01840014

### Assumptions
population = 350

prevalance = Layton, Salt Lake, and average % of UT tests coming back positive over the past week

### Question 1
If I use a single diagnostic test on my population of 350 people how many negative tests are likely false
- Test B has highest single test NPV

In Layton a person with a negative test has a 98-99% chance of being a true negative
- prevalance is 0.00052
- 0.1 people in my population are infected
- number of False Negatives = 0.003
    
In Salt Lake a person with a negative test has a 80-89% chance of being a true negative 
- prevalance is 0.00399
- 1.5 people in my population are infected
- number of False Negatives = 0.15
    
In All of UT a person with a negative test has a 97-99% chance of being a true negative
- prevalance is 0.045
- 16.1 people in my population are infected
- number of False Negatives = 3.2

### Question 2
If I use two tests in parallel on my population of 350 people how many negative tests are likely false
- Parallel OR has highest test NPV

In Layton a person with a negative test has a 98-99% chance of being a true negative
- prevalance is 0.00052
- 0.2 people in my population are infected
- number of False Negatives = 0.007
    
In Salt Lake a person with a negative test has a 80-89% chance of being a true negative 
- prevalance is 0.00399
- 1.4 people in my population are infected
- number of False Negatives = 0.059
    
In All of UT a person with a negative test has a 97-99% chance of being a true negative
- prevalance is 0.045
- 15.7 people in my population are infected
- number of False Negatives = 0.55


In [8]:
prev_list[2]

0.045

In [9]:
UT= Test_characteristics[Test_characteristics.Prevalance.eq(prev_list[2])]
Layton= Test_characteristics[Test_characteristics.Prevalance.eq(prev_list[0])]
Salt_Lake= Test_characteristics[Test_characteristics.Prevalance.eq(prev_list[1])]
Layton
Salt_Lake
UT

,Prevalance,Test Name,sensitivity,specificitiy,negPredVal
0,0.00052,Test A,0.63000,0.99000,0.999806
1,0.00052,Test B,0.84100,0.92300,0.999910
2,0.00052,Parallel OR,0.94117,0.91377,0.999967
3,0.00052,Parallel AND,0.52983,0.99923,0.999755
4,0.00052,Series OR,0.94117,0.91377,0.999967
5,0.00052,Series AND,0.52983,0.99923,0.999755


,Prevalance,Test Name,sensitivity,specificitiy,negPredVal
0,0.00415,Test A,0.63000,0.99000,0.998445
1,0.00415,Test B,0.84100,0.92300,0.999283
2,0.00415,Parallel OR,0.94117,0.91377,0.999732
3,0.00415,Parallel AND,0.52983,0.99923,0.998043
4,0.00415,Series OR,0.94117,0.91377,0.999732
5,0.00415,Series AND,0.52983,0.99923,0.998043


,Prevalance,Test Name,sensitivity,specificitiy,negPredVal
0,0.045,Test A,0.63000,0.99000,0.982694
1,0.045,Test B,0.84100,0.92300,0.991948
2,0.045,Parallel OR,0.94117,0.91377,0.996975
3,0.045,Parallel AND,0.52983,0.99923,0.978309
4,0.045,Series OR,0.94117,0.91377,0.996975
5,0.045,Series AND,0.52983,0.99923,0.978309


In [10]:
# ONE TEST - pick best NPV
# chance of being positive * chance of false negative*total pop
Layton_FN = pop*prev_list[0]*(1-max(Layton.negPredVal[0],Layton.negPredVal[1]))
SLC_FN = pop*prev_list[1]*(1-max(Salt_Lake.negPredVal[0],Salt_Lake.negPredVal[1]))
UT_FN = pop*prev_list[2]*(1-max(UT.negPredVal[0],UT.negPredVal[1]))
FP1 = [Layton_FN,SLC_FN,UT_FN]

In [11]:
# TWO TEST PARALLEL OR 
# chance of being positive * chance of false negative*total pop
Layton_FN2 = pop*prev_list[0]*(1-Layton.negPredVal[2])
SLC_FN2 = pop*prev_list[1]*(1-Salt_Lake.negPredVal[2])
UT_FN2 = pop*prev_list[2]*(1-UT.negPredVal[2])
FP2 = [Layton_FN2,SLC_FN2,UT_FN2]

In [12]:
plist = list(prev_list*2)
typelist = ['single','single','single','parallelOR','parallelOR','parallelOR']
infected = [pop*prev_list[0],pop*prev_list[1],pop*prev_list[2]]*2
FPlist = FP1+FP2
locations = ['Layton','Salt Lake','UT % test +','Layton','Salt Lake','UT % test +']

In [13]:
data_tuples2 = list(zip(locations, plist,typelist,infected,FPlist))
False_Pos = pd.DataFrame(data_tuples2,columns = ('Location','Prevalance','Test_Type','Number_Infected','Number_False_Positives'))

In [14]:
False_Pos
False_Pos.to_csv('FPs')

,Location,Prevalance,Test_Type,Number_Infected,Number_False_Positives
0,Layton,0.00052,single,0.1820,0.000016
1,Salt Lake,0.00415,single,1.4525,0.001042
2,UT % test +,0.04500,single,15.7500,0.126816
3,Layton,0.00052,parallelOR,0.1820,0.000006
4,Salt Lake,0.00415,parallelOR,1.4525,0.000390
5,UT % test +,0.04500,parallelOR,15.7500,0.047636
